In [ ]:
OPC_JOBS_TO_MONITOR=["FMDWI02_\_NZ",  # remember to add "escape '\' in sql 'like' conditions
                     "FMDWI02_\_NZ",
                     "FRAPX96_\_NZ",
                     "FLIKX01_\_NZ",
                     "FRAPX95_\_NZ",
                     "FRAPX94_\_NZ",
                     "FMDWI29_\_NZ",
                     "FKOF0940_NZ"]

date_range_condition="START_TIME >= '2019-11-01'"
#and START_TIME < '2020-01-10' "
                     
ftst2_where_condition=" OR ".join(map(lambda opc: f"[OPC_JOBNAME] LIKE '{opc}' ESCAPE '\\'" ,OPC_JOBS_TO_MONITOR))
                     
prod_where_condition=" OR ".join(map(lambda opc: f"[OPC_JOBNAME] LIKE '{opc}' ESCAPE '\\'" ,
                                     map(lambda opc: "P"+opc.replace("_NZ","")[1:-1]+"%", OPC_JOBS_TO_MONITOR)
                                     )
                                 )



sql_text_ftst2=f"select * from [D00000TD10_PWC_REP_FTST2].[dbo].[REP_WFLOW_RUN_INST]  WHERE {ftst2_where_condition} AND {date_range_condition}"
sql_text_prod=f"select * from [D00000PD10_PWC_REP_PROD].[dbo].[REP_WFLOW_RUN_INST]  WHERE {prod_where_condition} AND {date_range_condition}"

print(sql_text_ftst2)
DF_FTST2_REP_WFLOW_RUN_INST=t.context.DB_SQLFTST2_PC_REP.get_spark_dataframe(sql_text_ftst2)
print(sql_text_prod)
DF_PROD_REP_WFLOW_RUN_INST=t.context.DB_SQLPROD_PC_REP.get_spark_dataframe(sql_text_prod)

DF_FTST2_REP_WFLOW_RUN_INST.registerTempTable("DF_FTST2_REP_WFLOW_RUN_INST")
DF_PROD_REP_WFLOW_RUN_INST.registerTempTable("DF_PROD_REP_WFLOW_RUN_INST")

In [ ]:
print(DF_FTST2_REP_WFLOW_RUN_INST.columns)
print(DF_PROD_REP_WFLOW_RUN_INST.columns)

In [ ]:
t.spark.sql("""
with src as (select * ,  substring(workflow_name,1,length(workflow_name)-instr(reverse(workflow_name),'_')) workflow_name_short from DF_FTST2_REP_WFLOW_RUN_INST ) 
SELECT  workflow_name_short, CAST(START_TIME AS DATE) start_day, min(START_TIME) min_start_time, max(START_TIME) max_start_time, concat_ws(',', collect_list(OPC_JOBNAME)) OPC_JOBNAME_LIST  FROM src
GROUP BY workflow_name_short, CAST(START_TIME AS DATE)
order by workflow_name_short,start_day desc
""")

In [ ]:
t.spark.sql("""
with src as (select * ,  substring(workflow_name,1,length(workflow_name)-instr(reverse(workflow_name),'_')) workflow_name_short from DF_PROD_REP_WFLOW_RUN_INST 
where CAST(START_TIME AS DATE) > '2020-01-04'
) 
SELECT  workflow_name_short, CAST(START_TIME AS DATE) start_day, min(START_TIME) min_start_time, max(START_TIME) max_start_time, concat_ws(',', collect_list(OPC_JOBNAME)) OPC_JOBNAME_LIST  FROM src
GROUP BY workflow_name_short, CAST(START_TIME AS DATE)
order by workflow_name_short,start_day desc
""")

In [ ]:
# get recon errors
import pyetltools as t
ftst2_context=t.context.DB_NZFTST2
print(ftst2_context.sql.recon.aggregates_mdw_errors("%FACILITET_ULTIMO%","20200113"))

In [ ]:
recon_errors=ftst2_context.get_spark_dataframe(ftst2_context.sql.recon.aggregates_mdw_errors("%FACILITET_ULTIMO%","20200113"))

t.spark_helper.df_to_excel(recon_errors,r"c:\out\recon_errors.xls")
recon_errors.count()

In [ ]:
recon_errors=ftst2_context.get_spark_dataframe(ftst2_context.sql.recon.aggregates_mdw("%FACILITET_ULTIMO%","20200113"))

t.spark_helper.df_to_excel(recon_errors,r"c:\out\recon_all.xls")
recon_errors.count()

In [ ]:
ftst2_context.get_spark_dataframe(ftst2_context.sqls.recon_find_errors("%FACILITET_ULTIMO%","20200109"))

In [ ]:


sql_prod=t.context.DB_SQLPROD_MDW.get_spark_dataframe("select *  from D00032PD00_MDW_PROD.dbo.TMDW_FACILITET_GODK_SEGMENT_SCD");
sql_prod.persist();

In [ ]:
sql_prod.where(" ret_wf_run_i >= -99999999  and BANK_I in (-1,10034) AND BANK_I <> 10053").count()

In [ ]:
nz_prod=t.context.DB_NZFTST2.get_spark_dataframe("select * from FTST2_MDW.RAP.KMDW_FACILITET_GODK_SEGMENT_SCD where bank_i=10034");
nz_prod.persist();

In [ ]:
sql_prod.where("FACilitet_i=34133714")

In [ ]:
rest=compare_dataframes(nz_prod, sql_prod, ['FACILITET_I','START_PER_I'], mode=2)

In [ ]:
rest.where("both <> 1")

In [ ]:
sql_prod.write.save("c:\\out\\sql_prod.parquet")

In [ ]:
nz_prod.write.save("c:\\out\\nz_prod.parquet")

In [ ]:
t.context.DB_NZFTST2.exec_temp_access("UG_FTST_ALL_R","Recon")

In [ ]:
t.context.DB_NZFTST2.get_databases()

In [ ]:
wf_relations=t.context.DB_NZFTST2.get_spark_dataframe("select folder ||'.'|| object_name object_id, opc_jobname, folder,  object_name, mapping_name, db_type||'.'||table_name table_id, table_name, table_type, db_layer, db_type, overwrite  from mmdb.admin.wf_relations")



In [ ]:
t.spark_helper.df_to_csv(wf_relations, "c:\\out\\wf_relations.csv")

In [ ]:
c.NotebookApp.extra_static_paths